In [1]:
from collections import namedtuple

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
COLORS = px.colors.qualitative.Alphabet

In [3]:
class Task:
    Parameters = namedtuple('Parameters', 'O C D T A')

    def __init__(self, parameters) -> None:
        self.parameters = parameters

    def __str__(self) -> str:
        return str(self.parameters)


In [4]:
def read_tasks(filename):
    tasks = []

    with open(filename) as fp:
        n = fp.readline()
        n = int(n)

        for i in range(n):
            line = fp.readline()
            line = line.strip()
            params = [int(v) for v in line.split()]

            task = Task( Task.Parameters(*params) )
            tasks.append(task)

    return tasks

In [5]:
def read_traces(filename):
    traces = []

    with open(filename) as fp:
        traces = fp.read()
        traces = traces.split()

    traces = np.array(traces)
    return traces.astype(int)

In [6]:
def compute_H(tasks):
    periods = [task.parameters.T for task in tasks]
    H = (periods and np.lcm.reduce(periods)) or 0
    return H

In [7]:
def summarize_tasks(tasks):
    n = len(tasks)
    H = compute_H(tasks)
    theta_per_tick = 360 / H
    return n, H, theta_per_tick

In [8]:
def get_ticks(task, H, for_deadlines=False):
    ticks = []
    offset = task.parameters.O
    upper = H
    if for_deadlines:
        offset += task.parameters.D
        upper += task.parameters.D

    ticks = np.array( range(offset, upper, task.parameters.T) )
    return ticks % H

In [9]:
def plot_events(i, ticks, theta_per_tick, r=60, symbol=None, size=20):
    r = [r] * ticks.size
    theta = ticks * theta_per_tick
    scatter = go.Scatterpolar(r=r, theta=theta, mode='markers', showlegend=False,
                            marker=dict(color=COLORS[i], symbol=symbol, size=20) )

    return scatter

In [10]:
def plot_task_events(i, task, H, for_deadlines=False):
    theta_per_tick = 360 / H
    symbol = 'circle-x'
    r = 60
    
    if for_deadlines:
        symbol = 'star-diamond'
        r = 50

    ticks = get_ticks(task, H, for_deadlines=for_deadlines)
    scatter = plot_events(i, ticks, theta_per_tick, r, symbol)
    return scatter

In [27]:
def plot_trace(_id, traces, H):
    trace = np.where(traces[:H] == _id) [0]
    theta_per_tick = 360 / H
    r = [50] * trace.size
    theta = ((trace + 0.5) % H) * theta_per_tick

    bar = go.Barpolar(r=r, theta=theta, width=theta_per_tick, name=f'Task {_id}',
                    marker=dict(color=COLORS[_id-1], line_width=0.2, line_color='white'), 
                    opacity=0.8)

    return bar

In [28]:
tasks = read_tasks('data/example1.txt')
traces = read_traces('data/trace1.txt')

In [29]:
H = compute_H(tasks)
ticks = np.arange(H)

In [30]:
fig = go.Figure()

for i, task in enumerate(tasks[:]):
    scatter_releases = plot_task_events(i, task, H)
    scatter_deadlines = plot_task_events(i, task, H, for_deadlines=True)

    bar = plot_trace(i+1, traces, H)

    fig.add_trace(scatter_releases)
    fig.add_trace(scatter_deadlines)
    fig.add_trace(bar)

# Add cost trace
bar = plot_trace(0, traces, H)
fig.add_trace(bar)

fig.update_layout(template=None, autosize=False, width=800, height=800,
                polar=dict(angularaxis= dict(ticks='outside', tickmode='array', 
                                            tickvals=ticks * (360 / H), ticktext=ticks),
                        radialaxis = dict(showticklabels=False, ticks='') ) )

fig.show()